In [134]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
import os


In [26]:
load_dotenv()

True

In [184]:
MODEL_NAME = "llama-3.3-70b-versatile"

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name=MODEL_NAME
)

In [181]:
url = "https://finnair.wd103.myworkdayjobs.com/finnair/job/Vantaa/Operations-Analyst_R244936-1?source=LinkedIn"

laoder = WebBaseLoader(url)
jd_page_content = laoder.load()[-1]

prompt = PromptTemplate.from_template(
    
    """
    Your role here is to extract some key information from this job description
    and return them in plain JSON format.(this is very important. do not change the format)
    information to extract: [`overview`,`company_name`,`role_name`, `responsibilities`,`experience`, `hard_skills`,`soft_skills`, `application_deadline`, `contact_person`]

    Some helpful tips:
    if you get multiple values, separate them with a comma and put them in a list.
    if you are not able to extract any information, you assign "n/a" to that field. No preamble is needed. Do not include any other information oe verbose output.
    Be concise and to the point.
    overview is a short summary of the job. in 2/3 sentences.
    hard skills: any skills that is tool or technology related.
    soft skills: any skills that is not related hard skills.
    responsibilities: job specific responsibilities.
    casing of character should be standard each time.


    example output: [add curly braces]

        "overview": "This is a job overview",
        "company_name": "Company Name",
        "role_name": "Data Analyst",
        "experience": "2 years",
        "hard_skills": ["Python", "SQL"],
        "soft_skills": ["Teamwork", "Communication"],
        "responsibilities": "This is a responsibility",
        "application_deadline": "01/01/2022",
        "contact_person": [name, email, phone]
    
    

    This is a job description for a job.
    {page_content}
    """
)

chain = prompt | llm
jd_info = chain.invoke(input={'page_content': jd_page_content})

jd_json_parser = JsonOutputParser()
jd_json = jd_json_parser.parse(jd_info.content)

In [182]:
#loading cv

file_path = "/Users/shakibibnashameem/Dropbox/Curriculum Vitae (CV)/2024/StoraEnso - Data Scientist/Shakib CV.pdf"

pdf_loader = PyPDFLoader(file_path)
cv_pages = pdf_loader.load_and_split()


def remove_extra_whitespace(sentence):
    return ' '.join(sentence.split())

clean_pages = [remove_extra_whitespace(page.page_content) for page in cv_pages]
cv_content = "\n".join(clean_pages)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)


In [183]:
find_relevency = PromptTemplate.from_template(

    """ 
    Your task is to find the relevancy of the provided CV with the job information provided.
    Here is the cv: {CV}
    Here is the job information: {jd_info}

    and then write an email to the hiring mamager to express your interest in the job.
    and also mention the relevancy of your CV with the job description. 
    The email should be in a formal tone and should be concise and to the point.

    The email should be in English and have a professional formal tone.
    the email should have the following things :

    1. Greetings
    2. breif introduction of from the cv (for this keep it relevent to the job description)
    3. mention where the candidate can offer help to the company according to the job descriotion.
    4. express interest in the job.
    5. Questions about the job or the company.(add an appropriate question that you might have about the job or the company)
    6. closing remarks and thank you.

    Note that is is human to human communication. So, keep it professional and formal and humanly.
    """
)

chain = find_relevency | llm
email = chain.invoke(input={'CV': cv_content,
                            'jd_info': jd_json})

final_email = email.content

print(final_email)

Subject: Application for Operations Analyst Position at Finnair

Dear Mr. Teemu Halttula,

I hope this email finds you well. My name is Shakib Ibna Shameem Polock, and I am a highly motivated and detail-oriented professional with a strong background in data analysis and machine learning. As a Master's student in Business Analytics at the University of Oulu, I have developed a solid foundation in data-driven decision-making and analytics practices.

With over a year of experience in data engineering and machine learning, I am confident that my skills and expertise can support the Ground Operations team at Finnair. My experience in maintaining production planning data, ensuring data insights, and establishing analytics practices aligns with the job description. I am proficient in Power BI, SQL, and Microsoft Power Platform, which are essential tools for this role. Additionally, my experience in data visualization, data processing, and predictive modeling can help the team to make informe